In [ ]:
import tensorflow as tf

In [ ]:
dropout_rate = tf.placeholder('float')
learning_rate = 0.001 # 0.5 진동한다.
training_epochs = 15
batch_size = 100
display_step = 1

In [ ]:
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot=True)

In [ ]:
# X = trX.reshape(-1, 28, 28, 1)

X = tf.placeholder('float', [None, 28, 28, 1])
Y = tf.placeholder('float', [None, 10])
dropout_rate = tf.placeholder('float')

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # define filter
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
W4 = tf.Variable(tf.random_normal([128 * 4 * 4, 625], stddev=0.01))
W_O = tf.Variable(tf.random_normal([625, 10], stddev=0.01))

# padding: SAME = stride가 1일때 input과 동일한 크기의 output이 나오도록 padding을 붙인다.
l1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding='SAME') # [28, 28, 32]
# print(l1) # for test
l1 = tf.nn.relu(l1)
# print(l1) # for test
l1 = tf.nn.max_pool(l1, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')  # [14, 14, 32]
# print(l1) # for test

l2 = tf.nn.relu(tf.nn.conv2d(l1, W2, strides = [1, 1, 1, 1], padding='SAME')) # [14, 14, 64]
l2 = tf.nn.max_pool(l2, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') # [7, 7, 64]
#l2 = tf.nn.dropout(l2, dropout_rate)

l3 = tf.nn.relu(tf.nn.conv2d(l2, W3, strides = [1, 1, 1, 1], padding='SAME')) # [7, 7, 128]
l3 = tf.nn.max_pool(l3, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') # [4, 4, 128]

l3 = tf.reshape(l3, [-1, W4.get_shape().as_list()[0]])
#l3 = tf.nn.dropout(l3, dropout_rate)

l4 = tf.nn.relu(tf.matmul(l3, W4))
l4 = tf.nn.dropout(l4, dropout_rate)

y_ = tf.matmul(l4, W_O)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, Y))
train = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict = tf.argmax(y_, 1)


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(100):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        sess.run(train, feed_dict={X:trX[start:end], Y:trY[start:end], dropout_rate:0.8, })
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:256]
        
        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict,feed_dict={X:teX[test_indices], Y:teY[test_indices], dropout_rate:1.0})))

In [ ]:
sess.close()